In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Sep 23 01:27:06 2020

@author: aniruddha maiti
"""

import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt

# Part 1. Data Wrangling/Feature Engineering

In [2]:
# Read Data Files

# diagnoses : ICD9 codes for diagnoses 
diagnoses = pd.read_csv('diagnoses.csv') # diagnoses 600381 x 6

#clinical notes for the sample cohort 
notes_small_cohort_v2 = pd.read_csv('notes_small_cohort_v2.csv') # notes_small_cohort_v2 39367 x 11

# labs_cohort : laboratory results for each patient, at each charttime 
labs_cohort = pd.read_csv('labs_cohort.csv') # labs_cohort 9947103 x 6

# vitals_cohort_sirs :  vital signs for each patient at each charttime 
vitals_cohort_sirs = pd.read_csv('vitals_cohort_sirs.csv') # vitals_cohort_sirs 20633223 x 6

* variables: *
** Subject_id:** unique identifier of the subject
** Hadm_id: :** unique identifier of the admission, could be multiple admissions per subject
** Icustay_id:** unique identifier of the icu stay, possibly multiple of these per admission if a patient is transfer into and out of the icu multiple times in an admission 
** Charttime:** timestamp for the clinical observation 


In [28]:
# check diagnoses dataframe 
diagnoses.head() # 600381 x 6

,row_id,subject_id,hadm_id,seq_num,icd9_code,mimic_id
0,4,3,145834,1.0,389,335873220
1,5,3,145834,2.0,78559,335873221
2,6,3,145834,3.0,5849,335873222
3,7,3,145834,4.0,4275,335873223
4,8,3,145834,5.0,41071,335873224


In [30]:
len(notes_small_cohort_v2["subject_id"].unique())

1000

In [6]:
# check clinical notes
notes_small_cohort_v2.head() # 39367 x 11

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,note_text
0,44005,3,145834,2101-10-31,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2101-10-20**] Discharg...
1,94503,3,145834,2101-10-21,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: S/P Car...
2,94502,3,145834,2101-10-21,NaN,NaN,Echo,Report,NaN,NaN,PATIENT/TEST INFORMATION:\nIndication: Left ve...
3,272787,3,145834,2101-10-26,NaN,NaN,ECG,Report,NaN,NaN,Sinus rhythm\nP-R interval increased\nLate R w...
4,769043,3,145834,2101-10-24,2101-10-24 08:05:00,NaN,Radiology,CHEST (PORTABLE AP),NaN,NaN,[**2101-10-24**] 8:05 AM\n CHEST (PORTABLE AP)...


In [15]:
# check few variables that might be required later for SEPSIS identification
print("Category of notes : ", notes_small_cohort_v2.category.unique())
print("Description of notes : ", notes_small_cohort_v2.description.unique())

Category of notes :  ['Discharge summary' 'Echo' 'ECG' 'Radiology' 'Nursing/other' 'Nursing'
 'Physician ' 'General' 'Respiratory ' 'Nutrition' 'Social Work'
 'Case Management ' 'Pharmacy' 'Rehab Services' 'Consult']
Description of notes :  ['Report' 'CHEST (PORTABLE AP)' 'VIDEO OROPHARYNGEAL SWALLOW'
 'CT ABDOMEN W/O CONTRAST' 'LIVER OR GALLBLADDER US (SINGLE ORGAN)'
 'BILAT LOWER EXT VEINS' 'RENAL TRANSPLANT U.S.'
 'R RENAL TRANSPLANT U.S. RIGHT' 'R UNILAT LOWER EXT VEINS RIGHT'
 'ART EXT (REST ONLY)' 'O ABDOMEN (SUPINE ONLY) IN O.R.'
 'EMERG BILAT LOWER EXT VEINS' 'P RENAL TRANSPLANT U.S. PORT' 'Addendum'
 'CT HEAD W/O CONTRAST' 'P RENAL U.S. PORT' 'CT C-SPINE W/O CONTRAST'
 'MR HEAD W & W/O CONTRAST' 'CTA HEAD W&W/O C & RECONS'
 'MR HEAD W/ CONTRAST' 'CHEST (PRE-OP PA & LAT)'
 'CT STEREOTAXIS W/ CONTRAST' 'BONE SCAN' 'O CHEST (PORTABLE AP) IN O.R.'
 'T-SPINE' 'TRAUMA #2 (AP CXR & PELVIS PORT)' 'CT L-SPINE W/O CONTRAST'
 'CTA NECK W&W/OC & RECONS' 'CT T-SPINE W/O CONTRAST'
 'MR HEAD

In [12]:
# print sample note_text
for idx, row in notes_small_cohort_v2.iloc[0:10].iterrows():
    print("####################  subject id : {0}  ####################".format(row.subject_id))
    print(row.note_text)

####################  subject id : 3  ####################
Admission Date:  [**2101-10-20**]     Discharge Date:  [**2101-10-31**]

Date of Birth:   [**2025-4-11**]     Sex:  M

Service:  Medicine

CHIEF COMPLAINT:  Admitted from rehabilitation for
hypotension (systolic blood pressure to the 70s) and
decreased urine output.

HISTORY OF PRESENT ILLNESS:  The patient is a 76-year-old
male who had been hospitalized at the [**Hospital1 190**] from [**10-11**] through [**10-19**] of [**2101**]
after undergoing a left femoral-AT bypass graft and was
subsequently discharged to a rehabilitation facility.

On [**2101-10-20**], he presented again to the [**Hospital1 346**] after being found to have a systolic
blood pressure in the 70s and no urine output for 17 hours.
A Foley catheter placed at the rehabilitation facility
yielded 100 cc of murky/brown urine.  There may also have
been purulent discharge at the penile meatus at this time.

On presentation to the Emergency Department, the patient w

In [17]:
# check labs data
labs_cohort.head(n=10)

,subject_id,hadm_id,icustay_id,charttime,lab_id,valuenum
0,3,145834,211552,2101-10-20 16:40:00,ANION GAP,17.0
1,3,145834,211552,2101-10-20 16:40:00,BANDS,2.0
2,3,145834,211552,2101-10-20 16:40:00,BICARBONATE,25.0
3,3,145834,211552,2101-10-20 16:40:00,BUN,53.0
4,3,145834,211552,2101-10-20 16:40:00,CHLORIDE,99.0
5,3,145834,211552,2101-10-20 16:40:00,CREATININE,3.2
6,3,145834,211552,2101-10-20 16:40:00,GLUCOSE,91.0
7,3,145834,211552,2101-10-20 16:40:00,HEMATOCRIT,30.2
8,3,145834,211552,2101-10-20 16:40:00,HEMOGLOBIN,10.0
9,3,145834,211552,2101-10-20 16:40:00,INR,1.3


In [16]:
# check vital signs
vitals_cohort_sirs.head(n=10)

,subject_id,hadm_id,icustay_id,charttime,valuenum,vital_id
0,3,145834,211552,2101-10-20 19:15:00,217.0,SysBP
1,3,145834,211552,2101-10-20 19:30:00,151.0,HeartRate
2,3,145834,211552,2101-10-20 19:30:00,102.0,SysBP
3,3,145834,211552,2101-10-20 19:45:00,135.0,HeartRate
4,3,145834,211552,2101-10-20 19:45:00,94.0,SysBP
5,3,145834,211552,2101-10-20 20:00:00,143.0,HeartRate
6,3,145834,211552,2101-10-20 20:00:00,24.0,RespRate
7,3,145834,211552,2101-10-20 20:00:00,78.0,SysBP
8,3,145834,211552,2101-10-20 20:00:00,80.0,SysBP
9,3,145834,211552,2101-10-20 20:15:00,165.0,HeartRate


#### Extract the subject_id of the 1000 subjects with the smallest subject_ids  ####
We will first check the number of unique subject_ids and print aggregates for hadm_id and icd9_code

In [13]:
diagnoses[['subject_id', 'hadm_id','icd9_code']].groupby('subject_id').agg(['count', 'size', 'nunique'])

hadm_id              icd9_code             
             count size nunique     count size nunique
subject_id                                            
3                9    9       1         9    9       9
4                9    9       1         9    9       9
6                8    8       1         8    8       8
9                6    6       1         6    6       6
11               1    1       1         1    1       1
12               9    9       1         9    9       9
13               5    5       1         5    5       5
17              12   12       2        12   12      11
18              13   13       1        13   13      13
19               9    9       1         9    9       9
20               5    5       1         5    5       5
21              39   39       2        39   39      33
22               7    7       1         7    7       7
23              18   18       2        18   18      15
24               4    4       1         4    4       4
25               4    4       1         4    4       4
26               9    9       1         9    9       9
28              11   11       1        11   11      11
30               9    9       1         9    9       9
31               8    8       1         8    8       8
32               8    8       1         8    8       8
33               3    3       1         3    3       3
34              16   16       2        16   16      14
35              11   11       1        11   11      11
36              33   33       3        33   33      22
37               9    9       1         9    9       9
38              13   13       1        13   13      13
41              19   19       1        19   19      19
42               7    7       1         7    7       7
43               3    3       1         3    3       3
...            ...  ...     ...       ...  ...     ...
99894           13   13       1        13   13      13
99897           36   36       2        36   36      26
99899           27   27       1        27   27      27
99901           24   24       1        24   24      24
99903            5    5       1         5    5       5
99912           18   18       1        18   18      18
99913           12   12       1        12   12      12
99922            9    9       1         9    9       9
99923           31   31       2        31   31      28
99928            5    5       1         5    5       5
99934           11   11       1        11   11      11
99935           10   10       1        10   10      10
99936           11   11       1        11   11      11
99937           10   10       1        10   10      10
99938           13   13       1        13   13      13
99939            9    9       1         9    9       9
99944           16   16       1        16   16      16
99946           20   20       1        20   20      20
99955           11   11       1        11   11      11
99957           15   15       1        15   15      15
99965            5    5       1         5    5       5
99966           10   10       1        10   10      10
99973            9    9       1         9    9       9
99982           38   38       3        38   38      23
99983            8    8       1         8    8       8
99985           13   13       1        13   13      13
99991           17   17       1        17   17      17
99992           12   12       1        12   12      12
99995           17   17       1        17   17      17
99999            5    5       1         5    5       5

[38171 rows x 6 columns]

##### Select ids for sample subjects : extract unique subject_ids, then sort and select lowest 1000 #####

In [3]:
#sorted_diagnoses = diagnoses.sort_values(by = 'subject_id')

# extract unique subject ids from diagnoses
unique_subject_ids = diagnoses.subject_id.unique()
print("Number of patients in diagnoses: ", len(unique_subject_ids) )

# sort them
sorted_subject_ids = sorted(unique_subject_ids)

# take 1000 lowest subject ids as sample
sample_subjects = sorted_subject_ids[:1000]


Number of patients in diagnoses:  38171


##### Extract information about the sample ids only to reduce complexity of the subsequent tasks ######

In [4]:
sample_diagnoses = diagnoses.loc[diagnoses['subject_id'].isin(sample_subjects)]
print("Number of records in diagnoses for sample subjects: ", len(sample_diagnoses))

Number of records in diagnoses for sample subjects:  13889


In [6]:
# check how many subjects has at least one entry
len(sample_diagnoses.subject_id.unique())

1000

##### Do the same for all other dataframes #####

In [5]:
# clinical notes
sample_notes_small_cohort_v2 = notes_small_cohort_v2.loc[notes_small_cohort_v2['subject_id'].isin(sample_subjects)]
print("Number of notes for sample subjects: ", len(sample_notes_small_cohort_v2))

# lab results
sample_labs_cohort = labs_cohort.loc[labs_cohort['subject_id'].isin(sample_subjects)]
print("Number of records in labs cohort for sample subjects: ", len(sample_labs_cohort))

# vitals_cohort_sirs
sample_vitals_cohort_sirs = vitals_cohort_sirs.loc[vitals_cohort_sirs['subject_id'].isin(sample_subjects)]
print("Number of records in vital signs for sample subjects: ", len(sample_vitals_cohort_sirs))



Number of notes for sample subjects:  39367
Number of records in labs cohort for sample subjects:  295845
Number of records in vital signs for sample subjects:  599024


##### Check a few variables that are required for SIRS  iodentification #####

In [37]:
print("Lab Ids : ", sample_labs_cohort.lab_id.unique())
print("Vital Ids : ", sample_vitals_cohort_sirs.vital_id.unique())

# 1. The Systemic Inflammatory Response Syndrome (SIRS)

# print a few values of WBC, the given WBC condition is tricky
#print(sample_labs_cohort[sample_labs_cohort['lab_id']=='WBC'].valuenum)

Lab Ids :  ['ANION GAP' 'BANDS' 'BICARBONATE' 'BUN' 'CHLORIDE' 'CREATININE' 'GLUCOSE'
 'HEMATOCRIT' 'HEMOGLOBIN' 'INR' 'PLATELET' 'POTASSIUM' 'PT' 'PTT' 'SODIUM'
 'WBC' 'LACTATE' 'PaCO2' 'ALBUMIN' 'BILIRUBIN']
Vital Ids :  ['SysBP' 'HeartRate' 'RespRate' 'TempC']


#### Systemic Inflammatory Response Syndrome (SIRS) criteria ####

-	Temperature > 38°C or < 36°C ** Variable name ** TempC in vital
-   Heart Rate > 90 ** Variable name ** HeartRate in vital
-   Respiratory Rate > 20 or PaCO2 < 32mmHg ** Variable name ** RespRate in vital and PaCO2 in lab
-   WBC > 12,000/ mm3, < 4000/ mm3, or > 10% bands ** Variable name** WBC in lab


#### SEPSIS criteria ####
-	SIRS criteria + fixed number of ICD9 codes
-   3digitprefixes: '001','002','003','004','005','008',                 '009','010','011','012','013','014','015','016','017','018',                     '020','021','022','023','024','025','026','027','030','031',                     '032','033','034','035','036','037','038','039','040','041',                     '090','091','092','093','094','095','096','097','098','100',                     '101','102','103','104','110','111','112','114','115','116',                     '117','118','320','322','324','325','420','421','451','461',                    '462','463','464','465','481','482','485','486','494','510',                    '513','540','541','542','566','567','590','597','601','614',                   '615','616','681','682','683','686','730'
-   2digitprefixes: '5695','5720','5721','5750','5990','7110', '7907','9966','9985','9993'
-   5digit prefixes: '49121','56201','56203','56211','56213', '56983'

In [6]:
# List of sepsis ICD9 codes
sepsis_codes_3d = ['001','002','003','004','005','008',
                '009','010','011','012','013','014','015','016','017','018',
                '020','021','022','023','024','025','026','027','030','031',
                '032','033','034','035','036','037','038','039','040','041',
                '090','091','092','093','094','095','096','097','098','100',
                '101','102','103','104','110','111','112','114','115','116',
                '117','118','320','322','324','325','420','421','451','461',
                '462','463','464','465','481','482','485','486','494','510',
                '513','540','541','542','566','567','590','597','601','614', 
                '615','616','681','682','683','686','730']
sepsis_codes_4d = ['5695','5720','5721','5750','5990','7110', '7907','9966','9985','9993']
sepsis_codes_5d = ['49121','56201','56203','56211','56213', '56983']

##### SIRS and SEPSIS Status #####
###### Running Variables ######
As charttime data is too precise and there might be a lag between **lab data** and **vital data**, the safe way would be to save a running status of the variables responsible for SIRS. This will solve two problems at once, small change or mismatch in time-stamp would not affect the SIRS status, on the other hand, this approach also takes care of missing values. If at a certain timestamp, either lab data or vital data is missing, the previous value would be taken.

In [7]:
# for each subject, create the analytic data set
Patient_Timeline_data = pd.DataFrame(data=[], columns=['subject_id', 
                                                        'hadm_id', 'icustay_id', 'charttime', 'sirs', 'sepsis'])

# for each subject, create the analytic data set
for sub_id in sample_subjects[0:2]:
    print("##--------------------------##")
    print("      subject ID : ", sub_id)
    # select diagnoses
    diag = diagnoses[diagnoses['subject_id']==sub_id]
    vital = sample_vitals_cohort_sirs[sample_vitals_cohort_sirs['subject_id']==sub_id]
    lab = sample_labs_cohort[sample_labs_cohort['subject_id']==sub_id]
    patient_time_line = []
    #single_pat_data = pd.DataFrame(data=[], columns=['subject_id', 'hadm_id', 'icustay_id', 'charttime', 'sir', 'sepsis'])
    #result = pd.merge(vital, lab, on=['subject_id', 'hadm_id', 'icustay_id', 'charttime'], how='outer')
    
    # read patients data from vital and lab
    for row, v in vital.iterrows():
        #print(v)
        sir = 0 # dummy variable
        sepsis = 0 # dummy variable
        patient_time_line.append([sub_id, v.hadm_id, v.icustay_id, v.charttime, sir, sepsis])
    for row, lb in lab.iterrows():
        sir = 0 # dummy variable
        sepsis = 0 # dummy variable
        patient_time_line.append([sub_id, lb.hadm_id, lb.icustay_id, lb.charttime, sir, sepsis])
        
    # create dataframe to hold a single ptient data
    pat_data = pd.DataFrame(data=np.array(patient_time_line), columns=['subject_id', 
                                                        'hadm_id', 'icustay_id', 'charttime', 'sirs', 'sepsis'])
    # sort by chart time
    pat_data.sort_values(by=['charttime'])
    
    # extract unique time stamps
    unique_charttime = pat_data.charttime.unique()
    unique_charttime = sorted(unique_charttime)
    
    # ###############  Now determine SIR and sepsis status for each time stamp #########
    # ----------------------------------------------------------------------------------
    running_sirs_sepsis_status = [0,0]
    
    # define indicator variables (running)
    # advantage of running indicator is mismatch between lab and vital data could be solved to certain extent
    temp_C = False
    heart_rate = False
    resp_rate = False
    paco2 = False
    wbc = False
    bands = False
    
    sirs_indicators = [temp_C, heart_rate, resp_rate, paco2, wbc, bands]
    for chartt in unique_charttime:
        print(chartt)
        pat_info_t = pat_data[pat_data['charttime']==chartt]
        #print(pat_info_t)
        pat_hadm_id = pat_info_t.iloc[0].hadm_id
        pat_icu_id = pat_info_t.iloc[0].icustay_id
        

        # select the data corresponding to this time stamp
        selected_vital_data = vital[vital['charttime']==chartt]
        selected_lab_data = lab[lab['charttime']==chartt]
        
        # debug to cross verify if conditions are eally met
        #print("####### Vital data ###########")
        #print(selected_vital_data)
        #print("####### Lab data ###########")
        #print(selected_lab_data)

        # check vital data conditions
        for row, d in selected_vital_data.iterrows():
                    if d.vital_id == "TempC":
                        if (d.valuenum > 38.0) or (d.valuenum < 36.0):
                            temp_C = True
                        else:
                            temp_C = False

                    if d.vital_id == "HeartRate":   
                        if (d.valuenum > 90.0):
                            heart_rate = True
                        else:
                            heart_rate = False

                    if d.vital_id == "RespRate":   
                        if (d.valuenum > 20.0):
                            resp_rate = True 
                        else:
                            resp_rate = False 

        # check lab data conditions
        for row, d in selected_lab_data.iterrows():
                    if d.lab_id == "PaCO2":
                        if d.valuenum < 32.0 :
                            paco2 = True
                        else :
                            paco2 = False

                    if d.lab_id == "WBC": 
                        if (d.valuenum > 12000.0) or (d.valuenum < 4000.0):
                            wbc = True
                        else:
                            wbc = False
                        
                    if d.lab_id == "BANDS":    
                            if (d.valuenum < 10.0):
                                bands = True
                            else:
                                bands = False
                                
                                
        sirs_indicators = [temp_C, heart_rate, resp_rate, paco2, wbc, bands]
        #print(temp_C, heart_rate, resp_rate, paco2, wbc, bands)
        
        # Now evaluate conditions
        flag = 0
        if sirs_indicators[0]:
            flag = flag + 1
        if sirs_indicators[1]:
            flag = flag + 1
        if sirs_indicators[2] or sirs_indicators[3]:
            flag = flag + 1        
        if sirs_indicators[4] or sirs_indicators[5]:
            flag = flag + 1 
        
        # if at least two conditions met, declare sirs
        if flag >= 2:
            running_sirs_sepsis_status[0] = 1            
            pat_data.loc[pat_data['charttime']==chartt, 'sirs'] = 1
            
        
            # If SIRS condition is met then only check for sepsis status:
            
            # extract diagnosis codes for this hadm_id
            # this will be required for sepsis
            diagnosis_codes = list(diag[diag['hadm_id']==int(pat_hadm_id)].icd9_code) 
            for c in diagnosis_codes:
                if (c in sepsis_codes_3d) or (c in sepsis_codes_4d) or ((c in sepsis_codes_5d)):
                    running_sirs_sepsis_status[1] = 1
                    pat_data.loc[pat_data['charttime']==chartt, 'sepsis'] = 1
    
    # append to the dataframe containing timeline information about all 1000 sampled patients
    Patient_Timeline_data = pd.concat([Patient_Timeline_data, pat_data])
print(Patient_Timeline_data.head(n=100))
# save in csv format
Patient_Timeline_data.to_csv('analytic_dataset_all_1.csv')


##--------------------------##
      subject ID :  3
2101-10-20 16:40:00
2101-10-20 16:49:00
2101-10-20 19:12:00
2101-10-20 19:14:00
2101-10-20 19:15:00
2101-10-20 19:26:00
2101-10-20 19:30:00
2101-10-20 19:45:00
2101-10-20 19:59:00
2101-10-20 20:00:00
2101-10-20 20:04:00
2101-10-20 20:15:00
2101-10-20 20:30:00
2101-10-20 20:45:00
2101-10-20 21:00:00
2101-10-20 21:15:00
2101-10-20 21:30:00
2101-10-20 21:45:00
2101-10-20 21:51:00
2101-10-20 22:00:00
2101-10-20 22:15:00
2101-10-20 22:30:00
2101-10-20 22:45:00
2101-10-20 23:00:00
2101-10-20 23:15:00
2101-10-21 00:00:00
2101-10-21 00:42:00
2101-10-21 01:00:00
2101-10-21 01:46:00
2101-10-21 01:53:00
2101-10-21 02:00:00
2101-10-21 03:00:00
2101-10-21 03:09:00
2101-10-21 04:00:00
2101-10-21 04:15:00
2101-10-21 05:00:00
2101-10-21 06:00:00
2101-10-21 07:00:00
2101-10-21 07:05:00
2101-10-21 07:34:00
2101-10-21 08:00:00
2101-10-21 08:46:00
2101-10-21 08:51:00
2101-10-21 09:00:00
2101-10-21 10:15:00
2101-10-21 10:23:00
2101-10-21 10:45:00
2101-10

In [6]:
#Patient_Timeline_data.to_csv('analytic_dataset_all.csv')
# read data csv format
Patient_Timeline_data = pd.read_csv('analytic_dataset_all.csv')

###### Text based search for sepsis ######
In addition to ICD9 code based status assignment to possible sepsis cases, perform a text based search to designate a case as sepsis. If  there is at least one mention of **sepsis** or **septic** in a clinical note, then we will make the sepsis code as 1, irrespective of ICD9 based status.

As text search is a time consuming task, intentionally, we will not use fancy text processing. Clinical notes are more or less straightforward and it is relatively easy to find a word. Special characters unlikely to cause any problem. 
We will use pandas string methods to do it quickly and efficiently. Make all strings lowercase so avoid problems with capitalization.

In [7]:
for idx, row in Patient_Timeline_data.iterrows():
    # SIRS is a pre-condition for sepsis
    # check SIR first, if sepsis is already 1, no need to check the text
    if (row.sirs == 1) and (row.sepsis == 0):
        if idx % 50000 == 0:
            print(idx)
        sepsis_flag = sample_notes_small_cohort_v2[sample_notes_small_cohort_v2['hadm_id']==row.hadm_id]['note_text'].str.lower().str.contains('sepsis')
        septic_flag = sample_notes_small_cohort_v2[sample_notes_small_cohort_v2['hadm_id']==row.hadm_id]['note_text'].str.lower().str.contains('septic')
        if ((sepsis_flag.sum() + septic_flag.sum()) >=1):
            Patient_Timeline_data.loc[idx, 'sepsis'] = 1
            
            
Patient_Timeline_data.to_csv('analytic_dataset_all_with_text.csv')

0
250000
350000
400000
450000
550000
800000


3

A few Statistics : The number of unique SIRS and sepsis patients

In [8]:
print("Unique SIRS charttimes : ", sum(Patient_Timeline_data['sirs']==1))
print("Unique sepsis charttimes : ", sum(Patient_Timeline_data['sepsis']==1))

print("Unique SIRS subjects : ", len(Patient_Timeline_data[Patient_Timeline_data['sirs']==1].subject_id.unique()))
print("Unique sepsis subjects : ", len(Patient_Timeline_data[Patient_Timeline_data['sepsis']==1].subject_id.unique()))

Unique SIRS charttimes :  582147
Unique sepsis charttimes :  384474
Unique SIRS subjects :  980
Unique sepsis subjects :  418


In [15]:
Patient_Timeline_data[Patient_Timeline_data['sepsis']==1]

,Unnamed: 0,subject_id,hadm_id,icustay_id,charttime,sirs,sepsis
950,0,4,185777,294638,2191-03-16 01:30:00,1,1
951,1,4,185777,294638,2191-03-16 01:30:00,1,1
952,2,4,185777,294638,2191-03-16 02:00:00,1,1
953,3,4,185777,294638,2191-03-16 02:00:00,1,1
959,9,4,185777,294638,2191-03-16 06:00:00,1,1
960,10,4,185777,294638,2191-03-16 07:00:00,1,1
961,11,4,185777,294638,2191-03-16 07:00:00,1,1
962,12,4,185777,294638,2191-03-16 07:00:00,1,1
963,13,4,185777,294638,2191-03-16 08:00:00,1,1
964,14,4,185777,294638,2191-03-16 08:00:00,1,1


In [17]:
print(vital.iloc[:5])

   subject_id  hadm_id  icustay_id            charttime  valuenum   vital_id
0           3   145834      211552  2101-10-20 19:15:00     217.0      SysBP
1           3   145834      211552  2101-10-20 19:30:00     151.0  HeartRate
2           3   145834      211552  2101-10-20 19:30:00     102.0      SysBP
3           3   145834      211552  2101-10-20 19:45:00     135.0  HeartRate
4           3   145834      211552  2101-10-20 19:45:00      94.0      SysBP


# Part 2. Data

Read the datasets and print them to understand the variables

In [18]:
# Patient_feature_matrix.csv – the analytic dataset to use for this task
# patient_feature_matrix 3455 x 2400
patient_feature_matrix = pd.read_csv('patient_feature_matrix.csv') 

# Feature_descriptions.csv – descriptions of the features in 
# patient_feature_matrix and their provenance
# feature_descriptions 2438 x 4
feature_descriptions = pd.read_csv('feature_descriptions.csv') 

#Cohort.csv -- contains the index time, exposure time (if any), 
# in-hospital time of death (if any), and the time of censoring 
# (when the patient was released from the hospital)
# cohort 3455 x 8
cohort = pd.read_csv('cohort.csv')


#### Know the data : -> print ####

In [23]:
patient_feature_matrix.head()

,subject_id,death_in_stay,oxy_drop,gender,age_in_days,icd9_ ADENOID CYSTIC CARCINOMA/SDA,icd9_ INTRACTABLE EPILEPSY/SDA,icd9_?AORTIC DISSECTION,icd9_?TIA;IV HEPARIN,icd9_+ETT\CATH,...,chartvalue_815,chartvalue_821,chartvalue_824,chartvalue_825,chartvalue_828,chartvalue_829,chartvalue_833,chartvalue_837,chartvalue_861,chartvalue_87
0,91,died,stable,F,29809.000000,0,0,0,0,0,...,1.50000,1.700000,14.800000,42.400000,77.000000,3.900000,3.040000,144.000000,6.50000,15.0
1,106,died,oxy_drop,M,10358.333333,0,0,0,0,0,...,1.40027,2.600000,14.755973,35.507768,226.000000,3.700000,5.600000,136.000000,26.00000,11.0
2,111,survived,oxy_drop,F,24388.979167,0,0,0,0,0,...,1.40027,2.012763,12.200000,53.500000,208.493109,4.101856,3.535677,139.220795,12.65352,13.0
3,117,died,oxy_drop,F,18224.075000,0,0,0,0,0,...,1.90000,2.300000,17.200000,40.100000,42.000000,5.000000,2.310000,136.000000,7.00000,11.0
4,3,survived,oxy_drop,M,27951.375000,0,0,0,0,0,...,1.50000,2.300000,14.400000,36.200000,359.000000,4.400000,3.300000,138.000000,24.40000,11.0


In [20]:
feature_descriptions.head()

,feature,feature_type,code,description
0,icd9_.ADENOID.CYSTIC.CARCINOMA.SDA,icd9,.ADENOID.CYSTIC.CARCINOMA.SDA,.ADENOID.CYSTIC.CARCINOMA.SDA
1,icd9_.INTRACTABLE.EPILEPSY.SDA,icd9,.INTRACTABLE.EPILEPSY.SDA,.INTRACTABLE.EPILEPSY.SDA
2,icd9_.AORTIC.DISSECTION,icd9,.AORTIC.DISSECTION,.AORTIC.DISSECTION
3,icd9_.TIA.IV.HEPARIN,icd9,.TIA.IV.HEPARIN,.TIA.IV.HEPARIN
4,icd9_.ETT.CATH,icd9,.ETT.CATH,.ETT.CATH


In [21]:
cohort.head()

,subject_id,icustay_id,death_in_stay,deathtime,censor_time,index_time,exposure_time,oxy_drop
0,91,256972,died,2177-05-10 15:16:00,2177-05-10 15:16:00,2177-05-08 00:00:00,NaN,stable
1,106,252051,died,2192-08-15 21:15:00,2192-08-15 21:15:00,2192-08-10 08:00:00,2192-08-14 16:33:00,oxy_drop
2,111,254245,survived,NaN,2142-05-05 11:45:00,2142-04-24 23:30:00,2142-04-25 18:32:00,oxy_drop
3,117,217966,died,2133-12-01 17:00:00,2133-12-01 17:00:00,2133-11-20 01:48:00,2133-11-20 04:05:00,oxy_drop
4,3,211552,survived,NaN,2101-10-31 13:58:00,2101-10-21 09:00:00,2101-10-21 09:46:00,oxy_drop


In [22]:
# see feature types
feature_descriptions.feature_type.unique()

array(['icd9', 'chartindicator', 'chartvalue', 'note CUI', 'demographic',
       'engineered'], dtype=object)